# 제2 고지 : 자연스러운  코드로 
## STEP 19 : 변수 사용성 개선

DeZero를 좀 더 쉽게 사용할 수 있도록 개선하는 작업을 진행한다. 

그 첫번째로, `Variable` 클래스를 더욱 쉽게 사용할 수 있도록 한다. 

### 19.1 변수 이름 지정 
앞으로 수 많은 변수를 사용할 것이므로, 변수에 **이름**을 붙여 구분하기 쉽도록 개선한다. 




In [2]:
import numpy as np


class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        #########################
        self.name = name  # 변수 구분을 위한 `이름` 설정
        #########################
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

### 19.2 ndarray 인스턴스 변수 
`Variable` 은 데이터를 담는 `상자` 역할을 한다. 그러나 그 보다 중요한 것은 상자안의 `데이터` 이다. 그래서 `Variable` 이 데이터 처럼 보일 수 있게 하기 위해 작업한다.  
(여기서는 `ndarray` 만을 취급하기로 가정했기 때문에 `ndarray` 처럼 보이도록 작업을 한다.)  
필요한 더 많은 변수들을 추가할 수 있겠지만, 해당 예제에서는 다음 4가지 인스턴스 변수만 추가한다. 
1. shape
2. ndim
3. size
4. dtype 

In [3]:
import numpy as np


class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0
        
    ###############################
    @property
    def ndim(self):
        return self.data.ndim
    @property
    def shape(self):
        return self.data.shape
    @property
    def size(self):
        return self.data.size
    @property
    def dtype(self):
        return self.data.dtype
    ###############################

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None
        
    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [8]:
x = Variable(np.array([1,2,3]))
print(x.ndim)
print(x.shape)
print(x.size)
print(x.dtype)

1
(3,)
3
int64


### 19.3 len 함수와 print 함수
조금 더 확장하여, `len()`과 `print()` 또한 사용할 수 있도록 개선한다.

In [5]:
import numpy as np


class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def ndim(self):
        return self.data.ndim
    @property
    def shape(self):
        return self.data.shape
    @property
    def size(self):
        return self.data.size
    @property
    def dtype(self):
        return self.data.dtype

    #############################
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n","\n"+" "*9)
        return 'variable('+p+')'
        
    #############################
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None
        
    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [6]:
x = Variable(np.array([1,2,3]))
print(x)

variable([1 2 3])
